In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import torch
import numpy as np
from datetime import datetime

from faster_rcnn import network
from faster_rcnn.faster_rcnn import FasterRCNN, RPN
from faster_rcnn.utils.timer import Timer

import faster_rcnn.roi_data_layer.roidb as rdl_roidb
from faster_rcnn.roi_data_layer.layer import RoIDataLayer
from faster_rcnn.datasets.factory import get_imdb
from faster_rcnn.fast_rcnn.config import cfg, cfg_from_file

from faster_rcnn.pose_net import PoseNet

try:
    from termcolor import cprint
except ImportError:
    cprint = None

try:
    from pycrayon import CrayonClient
except ImportError:
    CrayonClient = None


def log_print(text, color=None, on_color=None, attrs=None):
    if cprint is not None:
        cprint(text, color=color, on_color=on_color, attrs=attrs)
    else:
        print(text)

In [30]:
# hyper-parameters
# ------------
#imdb_name = 'voc_2007_trainval'
imdb_name = 'kittipose_train'
cfg_file = 'experiments/cfgs/faster_rcnn_end2end.yml'
pretrained_model = '/home/pculbert/Documents/faster_rcnn_pytorch/VGG_imagenet.npy'
trained_fast_rcnn = 'models/saved_model9/faster_rcnn_100000.h5'
output_dir = 'models/saved_pose_model3'

start_step = 0
end_step = 80000
lr_decay_steps = {40000, 60000}
lr_decay = 1./10

rand_seed = 1024
_DEBUG = True
use_tensorboard = False
remove_all_log = False   # remove all historical experiments in TensorBoard
exp_name = None # the previous experiment name in TensorBoard

# ------------

if rand_seed is not None:
    np.random.seed(rand_seed)

In [31]:
# load config
cfg_from_file(cfg_file)
lr = cfg.TRAIN.LEARNING_RATE
momentum = cfg.TRAIN.MOMENTUM
weight_decay = cfg.TRAIN.WEIGHT_DECAY
disp_interval = cfg.TRAIN.DISPLAY
log_interval = cfg.TRAIN.LOG_IMAGE_ITERS

In [23]:
# load data
imdb = get_imdb(imdb_name)
rdl_roidb.prepare_roidb(imdb)
roidb = imdb.roidb
# roidb is a database, containing bounding box info for all training images
data_layer = RoIDataLayer(roidb, imdb.num_classes)

<bound method kittipose.default_roidb of <faster_rcnn.datasets.kittipose.kittipose object at 0x7f2940762e10>>
Remove empty annotations:  005066 004339 004040 003383 001752 001091 000547 Done. 
kittipose_train gt roidb loaded from /home/pculbert/Documents/faster_rcnn_pytorch/data/cache/kittipose_train_gt_roidb.pkl


In [24]:
# load net
net = PoseNet(classes=imdb.classes, debug=_DEBUG)
network.weights_normal_init(net, dev=0.01)
# network.load_pretrained_npy(net, pretrained_model)
network.load_net(trained_fast_rcnn, net.frcnn)
print('load model successfully!')

# model_file = '/media/longc/Data/models/VGGnet_fast_rcnn_iter_70000.h5'
# model_file = 'models/saved_model3/faster_rcnn_60000.h5'
# network.load_net(model_file, net)
# exp_name = 'vgg16_02-19_13-24'
# start_step = 60001
#lr /= 10.
# network.weights_normal_init([net.bbox_fc, net.score_fc, net.fc6, net.fc7], dev=0.01)

net.cuda()
net.train();

load model successfully!


In [7]:
#for name, param in net.named_parameters():
#    if param.requires_grad:
#        print name, param.data

In [8]:
#print list(net.parameters())[-4:]

In [8]:
lr = cfg.TRAIN.LEARNING_RATE
optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, net.parameters()), 
                            lr=lr, momentum=momentum, weight_decay=weight_decay)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# tensorboad
use_tensorboard = use_tensorboard and CrayonClient is not None
if use_tensorboard:
    cc = CrayonClient(hostname='127.0.0.1')
    if remove_all_log:
        cc.remove_all_experiments()
    if exp_name is None:
        exp_name = datetime.now().strftime('vgg16_%m-%d_%H-%M')
        exp = cc.create_experiment(exp_name)
    else:
        exp = cc.open_experiment(exp_name)
        
#for param in net.parameters():
#    print(type(param.data), param.size())

In [9]:
file_loss_pose = open(output_dir+'/loss_pose.txt', 'w')

train_loss = 0.
tp, tf, fg, bg = 0., 0., 0., 0.
step_cnt = 0.
re_cnt = False
t = Timer()
t.tic()
for step in range(start_step, end_step+1):
#for step in range(20):

    # get one batch
    blobs = data_layer.forward()
    im_data = blobs['data'] # one image, shape = (1, H, W, 3)
    im_info = blobs['im_info'] # stores H, W, scale
    gt_boxes = blobs['gt_boxes'] # groundtruth boxes
    gt_poses = blobs['gt_poses']
    gt_ishard = blobs['gt_ishard']
    dontcare_areas = blobs['dontcare_areas']
    dontcare_poses = blobs['dontcare_poses']
    disp_data = blobs['data_disp'] # disparity map

    # forward
    cls_prob, _, _, pose_pred = net(im_data, im_info, disp_data, gt_boxes, gt_poses, gt_ishard, dontcare_areas, dontcare_poses)
    loss = net.loss

    
    if _DEBUG:
        tp += float(net.tp)
        tf += float(net.tf)
        fg += net.fg_cnt
        bg += net.bg_cnt

    train_loss += loss.data[0]
    step_cnt += 1

    # backward
    optimizer.zero_grad()
    loss.backward()
    network.clip_gradient(net, 10.)
    optimizer.step()
    
    
    # log losses to file
    file_loss_pose.write(str(net.loss_pose.data.cpu().numpy()[0]))
    file_loss_pose.write('\n')
    

    #if step % disp_interval == 0:
    if step % 100 == 0:
        duration = t.toc(average=False)
        fps = step_cnt / duration

        log_text = 'step %d, image: %s, loss: %.4f, fps: %.2f (%.2fs per batch)' % (
            step, blobs['im_name'], train_loss / step_cnt, fps, 1./fps)
        log_print(log_text, color='green', attrs=['bold'])

        if _DEBUG:
            log_print('\tTP: %.2f%%, TF: %.2f%%, fg/bg=(%d/%d)' % (tp/float(fg*100.), tf/float(bg*100.), fg/float(step_cnt), bg/float(step_cnt)))
            log_print('\tpose_loss: %.4f' % (
                net.loss_pose.data.cpu().numpy())
            )
            #print 'truth: ' + str(blobs['gt_poses'][:,6])
            #print 'est: ' + str(pose_pred.data.cpu().numpy()[:,6])
        re_cnt = True

    if use_tensorboard and step % log_interval == 0:
        exp.add_scalar_value('train_loss', train_loss / step_cnt, step=step)
        exp.add_scalar_value('learning_rate', lr, step=step)
        if _DEBUG:
            exp.add_scalar_value('true_positive', tp/float(fg*100.), step=step)
            exp.add_scalar_value('true_negative', tf/float(bg*100.), step=step)
            losses = {'rpn_cls': float(net.rpn.cross_entropy.data.cpu().numpy()),
                      'rpn_box': float(net.rpn.loss_box.data.cpu().numpy()),
                      'rcnn_cls': float(net.cross_entropy.data.cpu().numpy()),
                      'rcnn_box': float(net.loss_box.data.cpu().numpy()),
                      'rcnn_pose': float(net.loss_pose.data.cpu().numpy())}
            exp.add_scalar_dict(losses, step=step)

    if (step % 5000 == 0) and step > 0:
        save_name = os.path.join(output_dir, 'faster_rcnn_{}.h5'.format(step))
        network.save_net(save_name, net)
        print('save model: {}'.format(save_name))
    if step in lr_decay_steps:
        lr *= lr_decay
        optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, net.parameters()), 
                                    lr=lr, momentum=momentum, weight_decay=weight_decay)

    if re_cnt:
        tp, tf, fg, bg = 0., 0., 0, 0
        train_loss = 0
        step_cnt = 0
        t.tic()
        re_cnt = False
        
file_loss_pose.close()

step 0, image: 004466.jpg, loss: 5.4381, fps: 6.78 (0.15s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(42/258)
	pose_loss: 5.4381
step 100, image: 004482.jpg, loss: 17.9767, fps: 7.68 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(43/256)
	pose_loss: 72.8074
step 200, image: 003219.jpg, loss: 17.5332, fps: 7.70 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(33/266)
	pose_loss: 8.7825
step 300, image: 002597.jpg, loss: 15.3112, fps: 7.77 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(36/263)
	pose_loss: 12.1756
step 400, image: 002434.jpg, loss: 18.2668, fps: 7.75 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(38/261)
	pose_loss: 8.8314
step 500, image: 001172.jpg, loss: 15.8727, fps: 7.64 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(44/255)
	pose_loss: 4.4695
step 600, image: 002623.jpg, loss: 17.8955, fps: 7.54 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(35/264)
	pose_loss: 1.1794
step 700, image: 000841.jpg, loss: 15.5714, fps: 7.62 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/

step 6300, image: 000006.jpg, loss: 15.2283, fps: 7.59 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(39/260)
	pose_loss: 19.8139
step 6400, image: 000705.jpg, loss: 14.7122, fps: 7.59 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(43/256)
	pose_loss: 22.2940
step 6500, image: 004524.jpg, loss: 18.8905, fps: 7.68 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(38/261)
	pose_loss: 19.9121
step 6600, image: 001567.jpg, loss: 12.0690, fps: 7.71 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(41/258)
	pose_loss: 2.5573
step 6700, image: 001247.jpg, loss: 16.7248, fps: 7.56 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(36/263)
	pose_loss: 22.6287
step 6800, image: 004549.jpg, loss: 17.2378, fps: 7.45 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(36/263)
	pose_loss: 9.6625
step 6900, image: 005295.jpg, loss: 15.1926, fps: 7.67 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(40/259)
	pose_loss: 17.2392
step 7000, image: 001954.jpg, loss: 13.6906, fps: 7.69 (0.13s per batch)
	TP: 0.01%, 

step 12600, image: 005541.jpg, loss: 13.4199, fps: 7.77 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(35/264)
	pose_loss: 17.2853
step 12700, image: 005154.jpg, loss: 12.4012, fps: 7.74 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(40/259)
	pose_loss: 2.9275
step 12800, image: 000513.jpg, loss: 13.7985, fps: 7.71 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(37/262)
	pose_loss: 4.4208
step 12900, image: 000771.jpg, loss: 13.9601, fps: 7.71 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(40/259)
	pose_loss: 12.2504
step 13000, image: 002394.jpg, loss: 12.8495, fps: 7.67 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(42/257)
	pose_loss: 4.4476
step 13100, image: 004565.jpg, loss: 12.7362, fps: 7.63 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(39/260)
	pose_loss: 2.4949
step 13200, image: 003308.jpg, loss: 14.5792, fps: 7.80 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(40/259)
	pose_loss: 24.9640
step 13300, image: 002196.jpg, loss: 14.2217, fps: 7.72 (0.13s per batch)
	TP: 0

step 18800, image: 005319.jpg, loss: 13.2033, fps: 7.70 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(39/260)
	pose_loss: 19.9116
step 18900, image: 000860.jpg, loss: 14.9460, fps: 7.70 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(42/257)
	pose_loss: 24.1472
step 19000, image: 004451.jpg, loss: 11.8293, fps: 7.73 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(39/260)
	pose_loss: 12.0375
step 19100, image: 002416.jpg, loss: 15.5257, fps: 7.76 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(36/263)
	pose_loss: 21.2857
step 19200, image: 000270.jpg, loss: 13.3861, fps: 7.84 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(38/261)
	pose_loss: 3.8870
step 19300, image: 005119.jpg, loss: 12.6034, fps: 7.78 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(42/257)
	pose_loss: 3.6933
step 19400, image: 005613.jpg, loss: 12.9960, fps: 7.73 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(35/264)
	pose_loss: 5.7405
step 19500, image: 004232.jpg, loss: 12.9354, fps: 7.68 (0.13s per batch)
	TP: 

step 25000, image: 003894.jpg, loss: 12.7184, fps: 7.68 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(37/262)
	pose_loss: 4.2601
save model: trained_models/saved_pose_model_final/faster_rcnn_25000.h5
step 25100, image: 002812.jpg, loss: 13.0727, fps: 7.76 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(43/256)
	pose_loss: 13.3790
step 25200, image: 000071.jpg, loss: 15.0774, fps: 7.66 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(40/259)
	pose_loss: 1.0637
step 25300, image: 001047.jpg, loss: 14.4233, fps: 7.75 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(40/259)
	pose_loss: 68.2455
step 25400, image: 002941.jpg, loss: 13.1821, fps: 7.75 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(37/262)
	pose_loss: 16.9843
step 25500, image: 000320.jpg, loss: 14.1384, fps: 7.68 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(38/261)
	pose_loss: 6.1044
step 25600, image: 003320.jpg, loss: 14.1820, fps: 7.74 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(42/257)
	pose_loss: 6.1653
step 2570

...

step 49800, image: 000618.jpg, loss: 10.0953, fps: 7.74 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(40/259)
	pose_loss: 0.9423
step 49900, image: 000310.jpg, loss: 10.2077, fps: 7.75 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(37/262)
	pose_loss: 3.7569
step 50000, image: 005557.jpg, loss: 12.2970, fps: 7.72 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(36/263)
	pose_loss: 5.8495
save model: trained_models/saved_pose_model_final/faster_rcnn_50000.h5
step 50100, image: 000693.jpg, loss: 9.5966, fps: 7.68 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(38/261)
	pose_loss: 21.1560
step 50200, image: 005241.jpg, loss: 10.7999, fps: 7.69 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(41/258)
	pose_loss: 6.1011
step 50300, image: 005052.jpg, loss: 10.7212, fps: 7.64 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(41/258)
	pose_loss: 4.3963
step 50400, image: 000715.jpg, loss: 9.0875, fps: 7.65 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(37/262)
	pose_loss: 13.4928
step 50500, 

step 56000, image: 005668.jpg, loss: 9.9516, fps: 7.56 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(38/261)
	pose_loss: 4.4732
step 56100, image: 004034.jpg, loss: 8.6760, fps: 7.65 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(36/263)
	pose_loss: 1.3207
step 56200, image: 001290.jpg, loss: 12.1823, fps: 7.62 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(36/263)
	pose_loss: 4.0499
step 56300, image: 003069.jpg, loss: 9.9722, fps: 7.68 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(36/263)
	pose_loss: 2.9991
step 56400, image: 002368.jpg, loss: 9.7365, fps: 7.64 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(40/259)
	pose_loss: 25.2304
step 56500, image: 004000.jpg, loss: 12.8814, fps: 7.69 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(37/262)
	pose_loss: 0.6554
step 56600, image: 003653.jpg, loss: 12.1282, fps: 7.75 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(39/260)
	pose_loss: 4.9068
step 56700, image: 003668.jpg, loss: 9.9401, fps: 7.69 (0.13s per batch)
	TP: 0.01%, T

step 62200, image: 000352.jpg, loss: 9.0271, fps: 7.74 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(37/262)
	pose_loss: 2.8140
step 62300, image: 001693.jpg, loss: 10.6588, fps: 7.70 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(37/262)
	pose_loss: 21.3749
step 62400, image: 005805.jpg, loss: 9.3365, fps: 7.65 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(41/258)
	pose_loss: 15.8080
step 62500, image: 003477.jpg, loss: 9.2281, fps: 7.72 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(35/264)
	pose_loss: 26.2043
step 62600, image: 001552.jpg, loss: 10.9917, fps: 7.76 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(33/266)
	pose_loss: 5.5915
step 62700, image: 004306.jpg, loss: 10.0475, fps: 7.61 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(35/264)
	pose_loss: 6.8726
step 62800, image: 003273.jpg, loss: 9.7038, fps: 7.74 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(41/258)
	pose_loss: 3.0777
step 62900, image: 003683.jpg, loss: 9.8913, fps: 7.76 (0.13s per batch)
	TP: 0.01%,

step 68400, image: 001449.jpg, loss: 9.3741, fps: 7.74 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(36/263)
	pose_loss: 55.4313
step 68500, image: 002910.jpg, loss: 10.4362, fps: 7.81 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(39/260)
	pose_loss: 4.0599
step 68600, image: 000930.jpg, loss: 11.6800, fps: 7.74 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(38/261)
	pose_loss: 32.0398
step 68700, image: 001520.jpg, loss: 11.1393, fps: 7.79 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(40/259)
	pose_loss: 3.9565
step 68800, image: 001967.jpg, loss: 11.2017, fps: 7.77 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(36/263)
	pose_loss: 10.8522
step 68900, image: 005112.jpg, loss: 11.3767, fps: 7.77 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(33/266)
	pose_loss: 3.4719
step 69000, image: 004184.jpg, loss: 10.3153, fps: 7.80 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(34/265)
	pose_loss: 20.4632
step 69100, image: 000266.jpg, loss: 12.0955, fps: 7.59 (0.13s per batch)
	TP: 0

step 74600, image: 000945.jpg, loss: 11.0986, fps: 7.69 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(47/252)
	pose_loss: 3.9089
step 74700, image: 005315.jpg, loss: 8.7692, fps: 7.69 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(43/256)
	pose_loss: 26.4421
step 74800, image: 001639.jpg, loss: 12.2184, fps: 7.69 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(36/263)
	pose_loss: 9.5786
step 74900, image: 001481.jpg, loss: 9.9826, fps: 7.77 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(38/261)
	pose_loss: 6.5573
step 75000, image: 005874.jpg, loss: 9.5731, fps: 7.68 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(36/263)
	pose_loss: 10.2443
save model: trained_models/saved_pose_model_final/faster_rcnn_75000.h5
step 75100, image: 001947.jpg, loss: 10.2839, fps: 7.62 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(36/263)
	pose_loss: 1.3955
step 75200, image: 000306.jpg, loss: 9.5142, fps: 7.76 (0.13s per batch)
	TP: 0.01%, TF: 0.01%, fg/bg=(45/254)
	pose_loss: 9.9305
step 75300, im

In [43]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


log_dir = "trained_models/saved_pose_model_final"
file_pose_loss = np.loadtxt(log_dir+"/loss_pose.txt")
plt.plot(file_pose_loss)

UnicodeEncodeError: 'decimal' codec can't encode characters in position 0-72944: invalid decimal Unicode string